Import all required python modules.

In [1]:
import cf_units
import cftime

In [2]:
import iris
import numpy as np

In [3]:
from iris.util import unify_time_units

In [4]:
from iris.util import equalise_attributes

In [5]:
from pathlib import Path

In [6]:
import matplotlib.pyplot as plt
import iris.quickplot as qplt

In [7]:
iris.FUTURE.date_microseconds = True

In [8]:
iris.FUTURE.save_split_attrs = True

Obtain the pressure data from spinup files for 2091.

In [9]:
filepaths = []
for month in range(1,13):
    spinup_filepath = Path(f'/g/data/p73/archive/CMIP7/ACCESS-ESM1-6/spinup/June2025/output999/atmosphere/netCDF/aiihca.pa-2091{month:02d}_mon.nc')
    filepaths.append(spinup_filepath)

In [10]:
spinup_cube_list = iris.load(filepaths)

In [11]:
pressure_cube_list = spinup_cube_list.extract(iris.Constraint(name='air_pressure'))

We now have a list of pressure cubes, one for each month of 2091.

In [12]:
for cube in pressure_cube_list:
    print(cube.coord('time'))

DimCoord :  time / (days since 0001-01-01 00:00, proleptic_gregorian calendar)
    points: [2091-01-16 12:00:00]
    bounds: [[2091-01-01 00:00:00, 2091-02-01 00:00:00]]
    shape: (1,)  bounds(1, 2)
    dtype: float64
    standard_name: 'time'
    var_name: 'time'
DimCoord :  time / (days since 0001-01-01 00:00, proleptic_gregorian calendar)
    points: [2091-02-15 00:00:00]
    bounds: [[2091-02-01 00:00:00, 2091-03-01 00:00:00]]
    shape: (1,)  bounds(1, 2)
    dtype: float64
    standard_name: 'time'
    var_name: 'time'
DimCoord :  time / (days since 0001-01-01 00:00, proleptic_gregorian calendar)
    points: [2091-03-16 12:00:00]
    bounds: [[2091-03-01 00:00:00, 2091-04-01 00:00:00]]
    shape: (1,)  bounds(1, 2)
    dtype: float64
    standard_name: 'time'
    var_name: 'time'
DimCoord :  time / (days since 0001-01-01 00:00, proleptic_gregorian calendar)
    points: [2091-04-16 00:00:00]
    bounds: [[2091-04-01 00:00:00, 2091-05-01 00:00:00]]
    shape: (1,)  bounds(1, 2)
  

In [13]:
pressure_cube_list.concatenate_cube()

ConcatenateError: failed to concatenate into a single cube.
  Cube metadata differs for phenomenon: air_pressure

When we try to concatenate these cubes, Iris complains that "Cube metadata differs for phenomenon: air_pressure"

In [14]:
removed_attributes = equalise_attributes(pressure_cube_list)

In [15]:
unify_time_units(pressure_cube_list)

In [16]:
new_cube_list = pressure_cube_list.concatenate()

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/_concatenate.py:1283: IrisUserWarning: Auxiliary coordinates are unequal for phenomenon `air_pressure`:
a: _CoordAndDims(coord=<AuxCoord: surface_altitude / (m)  <lazy>  shape(145, 192)>, dims=(2, 3))
b: _CoordAndDims(coord=<AuxCoord: surface_altitude / (m)  <lazy>  shape(145, 192)>, dims=(2, 3))
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/_concatenate.py:1283: IrisUserWarning: Found cubes with overlap on concatenate axis 0, skipping concatenation for these cubes
  warnings.warn(
/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/iris/_concatenate.py:1283: IrisUserWarning: Auxiliary coordinates are unequal for phenomenon `air_pressure`:
a: _CoordAndDims(coord=<AuxCoord: surface_altitude / (m)  <lazy>  shape(145, 192)>, dims=(2, 3))
b: _CoordAndDims(coord=<AuxCoord: surface_altitude / (m)  <lazy>  shape

We have now removed unequal attributes and unified time units, but Iris now complains that the `surface_altitude` coordinate are unequal.

In [17]:
print(new_cube_list[0].coord('surface_altitude').points - new_cube_list[1].coord('surface_altitude').points)

[[ 4.54747351e-13  4.54747351e-13  4.54747351e-13 ...  4.54747351e-13
   4.54747351e-13  4.54747351e-13]
 [ 1.09139364e-11 -1.13686838e-11 -1.22781785e-11 ...  1.36424205e-11
   1.04591891e-11 -1.18234311e-11]
 [ 3.63797881e-12 -1.04591891e-11 -2.72848411e-12 ... -3.18323146e-12
  -1.04591891e-11  1.13686838e-11]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


Checking, we see that the `surface_altitude` coordinates differ on the order of rounding error.

In [18]:
for cube in new_cube_list:
    print(cube.coord('time'))

DimCoord :  time / (days since 0001-01-01 00:00, proleptic_gregorian calendar)
    points: [
        2091-01-16 12:00:00, 2091-03-16 12:00:00, 2091-05-16 12:00:00,
        2091-07-16 12:00:00, 2091-08-16 12:00:00, 2091-10-16 12:00:00,
        2091-12-16 12:00:00]
    bounds: [
        [2091-01-01 00:00:00, 2091-02-01 00:00:00],
        [2091-03-01 00:00:00, 2091-04-01 00:00:00],
        [2091-05-01 00:00:00, 2091-06-01 00:00:00],
        [2091-07-01 00:00:00, 2091-08-01 00:00:00],
        [2091-08-01 00:00:00, 2091-09-01 00:00:00],
        [2091-10-01 00:00:00, 2091-11-01 00:00:00],
        [2091-12-01 00:00:00, 2092-01-01 00:00:00]]
    shape: (7,)  bounds(7, 2)
    dtype: float64
    standard_name: 'time'
    var_name: 'time'
DimCoord :  time / (days since 0001-01-01 00:00, proleptic_gregorian calendar)
    points: [2091-02-15 00:00:00]
    bounds: [[2091-02-01 00:00:00, 2091-03-01 00:00:00]]
    shape: (1,)  bounds(1, 2)
    dtype: float64
    standard_name: 'time'
    var_name: 'ti

In [19]:
for cube in new_cube_list:
    time_bounds = cube.coord('time').bounds
    print(time_bounds[:,1] - time_bounds[:,0])

[31. 31. 31. 31. 31. 31. 31.]
[28.]
[30. 30. 30. 30.]


When we examine time and time bounds, we see that the new cube list is split into groups of months with 31, 30 and 28 days. Why this corresponds to different surface heights is a mystery.

In [20]:
surface_altitude = new_cube_list[0].coord('surface_altitude')
for cube in new_cube_list[1:]:
    cube.remove_coord(cube.coord('surface_altitude'))
    cube.add_aux_coord(surface_altitude,data_dims=(2,3))

In [21]:
print(new_cube_list[0].coord('surface_altitude').points - new_cube_list[1].coord('surface_altitude').points)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
new_cube_list[0]

Air Pressure (Pa),time,model_level_number,latitude,longitude
Shape,7,38,145,192
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
latitude,-,-,x,-
longitude,-,-,-,x
Auxiliary coordinates,,,,
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-
surface_altitude,-,-,x,x


In [23]:
new_cube_list[1]

Air Pressure (Pa),time,model_level_number,latitude,longitude
Shape,1,38,145,192
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
latitude,-,-,x,-
longitude,-,-,-,x
Auxiliary coordinates,,,,
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-
surface_altitude,-,-,x,x


When we try to fix surface altitude by replacing it with the surface altitude from cube 0, this somehow destroys the dependence of the derived altitude coordinates on latitude and longitude dimensions.

In [24]:
new_new_cube_list = new_cube_list.concatenate()

ValueError: None is not in list

This prevents concatenation.